### Script a ser utilizado para retirar informação de um ficheiro genbank diretamente da web, sem ser necessário realizar download do mesmo ###

In [31]:
import mysql.connector as SQLC

DataBase = SQLC.connect(
   host ="geo.di.uminho.pt",
   user ="bioinformatica",
   password ="20221207",
    database ="grupo_PRJ"
)
Cursor = DataBase.cursor()

In [32]:
from Bio import SeqIO
from Bio import Entrez

In [33]:
#é necessário introduzir o email (Entrez.email = '') e introduzir o accession number do ficheiro a estudar (id = '')

Entrez.email = '********@alunos.uminho.pt'
handle = Entrez.efetch(db="nucleotide", id='*********', rettype="gb", retmode="text")
record = SeqIO.read(handle, 'gb')

### Tabela Locus ###

In [34]:
accession= record.id

locus_name = record.name

molecule = record.annotations['molecule_type']

segment_type = record.annotations['topology']

genbank_date = record.annotations['date']

genbank_division = record.annotations['data_file_division']


sql_locus = f"INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('{accession}', '{locus_name}', '{molecule}', '{segment_type}', '{genbank_date}', '{genbank_division}')"

print(sql_locus)

Cursor.execute(sql_locus)

DataBase.commit()

INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('L42024.1', 'HUMHLABH', 'mRNA', 'linear', '24-MAY-1995', 'PRI')


### Tabela sequences ###

In [35]:
seq = record.seq

seq_length = len(seq)

locus_accession = accession

sql_sequences = f"INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('{seq}', {seq_length}, '{locus_accession}')"

print(sql_sequences)

Cursor.execute(sql_sequences)
DataBase.commit()

INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('TCTAGAGAAGCCAATCAGCGTCGCCGGGGTCCCAGTTCTAAAGTCCCCACGCACCCACCCGGACTCAGAGTCTCCTCAGACGCGGAGATGCTGGTCATGGCGCCCCGAACCGTCCTCCTGCTGCTCTCGGCGGCCCTGGCCCTGACCGAGACCTGGGCCGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCTCAGTGGGCTACGTGGACGACACGCAGTTCGTGAGGTTCGACAGCGACGCCGCGAGTCCGAGAGAGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAATATTGGGACCGGAACACACAGATCTGCAAGACCAACACACAGACTGACCGAGAGAGCCTGCGGAACCTGCGCGGCTACTACAACCAGAGCGAGGCCGGGTCTCACACTTGGCAGACGATGTACGGCTGCGACGTGGGGCCGGACGGGCGCCTCCTCCGCGGGCATAACCAGTTCGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGAGCTCCTGGACCGCGGCGGACACCGCGGCTCAGATCACCCAGCGCAAGTGGGAGGCGGCCCGTGTGGCGGAGCAGCTGAGAACCTACCTGGAGGGCACGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCGCGGACCCCCCAAAGACACATGTGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGCGAGGACCAAACTCAGGACACCGAGCTTGTGGAGACCAGACCAGCAGGAGACAGAACCTTCCAGAAGTGGGCAGCTGTGGTGGTGCCTTCTGGAGAAGAGCAGAGATACACAT

### Tabela definition ###


In [36]:
definition = record.description

locus_accession = accession

sql_definition = f"INSERT INTO definition (definition, locus_accession) VALUES ('{definition}', '{locus_accession}')"

print(sql_definition)

Cursor.execute(sql_definition)

DataBase.commit()

INSERT INTO definition (definition, locus_accession) VALUES ('Homo sapiens MHC HLA-B39 mRNA, complete cds', 'L42024.1')


### Tabela locus_keywords ###

In [37]:
keyword_dict = {}

keywords_name = record.annotations['keywords']

for num, keyword in enumerate(keywords_name):
    if keywords_name == ['']:
        print('Este ficheiro não tem keywords associadas')
    else:
        sql_locus_keywords = f"INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('{locus_accession}', {num+1}, '{keyword}')"
        Cursor.execute(sql_locus_keywords)
        DataBase.commit()
        print(sql_locus_keywords)

INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 1, 'B*39011 allele')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 2, 'HLA-B39')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 3, 'cell surface antigen')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 4, 'cell surface glycoprotein')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 5, 'class I gene')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 6, 'integral membrane protein')
INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('L42024.1', 7, 'major histocompatibility complex')


### Tabela source ###

In [38]:
scientific_name = record.annotations['source']

sql_source = f"INSERT INTO source (scientific_name, locus_accession) VALUES ('{scientific_name}', '{locus_accession}')"

print(sql_source)

Cursor.execute(sql_source)

DataBase.commit()

INSERT INTO source (scientific_name, locus_accession) VALUES ('Homo sapiens (human)', 'L42024.1')


### Tabela taxonomy ###


In [39]:
taxonomy = record.annotations['taxonomy']

sql_taxonomy = f"INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('{'; '.join(taxonomy)}', '{locus_accession}')"

print(sql_taxonomy)

Cursor.execute(sql_taxonomy)

DataBase.commit()

INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae; Homo', 'L42024.1')


### Tabela reference ###

In [40]:
title = list()
journal = list()
pubmed_id = list()
ref_location = list()


for reference in record.annotations['references']:
    title.append(reference.title)
    journal.append(reference.journal)
    pubmed_id.append(reference.pubmed_id)
    for loc in reference.location:
        ref_location.append(loc)

    
for i in zip(title, journal, pubmed_id, ref_location):
    if i[2] == '' and i[3] == '':
        sql_references = f"INSERT INTO reference (title, journal, locus_accession) VALUES ('{i[0]}', '{i[1]}', '{locus_accession}')"
        print(sql_references)
        print('Esta referência não possui ID do pubmed, nem descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] == '' and i[3] != '':
        sql_references = f"INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ('{i[0]}', '{i[1]}', '{locus_accession}', '{i[3]}')"
        print(sql_references)
        print('Esta referência não possui ID do pubmed')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] != '' and i[3] == '':
        sql_references = f"INSERT INTO reference (title, journal, pubmed_id, locus_accession) VALUES ('{i[0]}', '{i[1]}', '{i[2]}', '{locus_accession}')"
        print(sql_references)
        print('Esta referência não descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()
        
    else:
        sql_references = f"INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('{i[0]}', '{i[1]}', '{i[2]}', '{locus_accession}', '{i[3]}')"
        print(sql_references)
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ('Full cDNA sequence of a novel HLA-B39 variant', 'Unpublished', 'L42024.1', '[0:1357]')
Esta referência não possui ID do pubmed



### Tabela pubmed ###

In [41]:
for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()
        for info in pubmed_record:
            title_pubmed = list()
            pub_date = list()
            pub_source = list()
            pub_issue = list()
            pub_pages = list()
            doi = list()

            title_pubmed.append(info['Title'])
            pub_date.append(info['PubDate'])
            pub_source.append(info['Source'])
            pub_issue.append(info['Issue'])
            pub_pages.append(info['Pages'])
            doi.append(info['DOI'])
        
        for i in zip(title_pubmed, pub_date, pub_source, pub_issue, pub_pages, doi):
            sql_pubmed = f"INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES ({id}, '{i[0]}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}')"
            print(sql_pubmed)    
            Cursor.execute(sql_pubmed)
            DataBase.commit()
            
    else:
        print('Sem resultados')


Sem resultados


### Tabela authors ###

In [42]:
author_names = dict()

for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()

        for info in pubmed_record:
            author_names[id] = info['AuthorList']
            
        for id, authors in author_names.items():
            for name in authors:
                sql_authors = f"INSERT INTO authors (author_name, pubmed_id) VALUES ('{name}', {id})"
                print(sql_authors)
                Cursor.execute(sql_authors)
                DataBase.commit()
    else:
        print('Sem resultados')
               

Sem resultados


### Tabela feat_source ###

In [43]:
features = record.features

for i in features:
    if i.type == 'source':
        location = set(i for i in i.location)
        organism = set(i for i in i.qualifiers['organism'])
        db_xref = set(i.qualifiers['db_xref'])

        sql_feat_source = f'INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("{i.location}", "{organism}", "{db_xref}", "{locus_accession}")'
        print(sql_feat_source)
        Cursor.execute(sql_feat_source)
        DataBase.commit()
        


INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("[0:1357](+)", "{'Homo sapiens'}", "{'taxon:9606'}", "L42024.1")


### Tabela feat_cds ###

In [44]:
location_list = list()
codon_start_list = list()
product_list = list()
protein_id_list = list()
translation_list = list()

for i in features:
    if i.type == 'CDS':
            
        location_list.append(i.location)
        codon_start_list.append(i.qualifiers['codon_start'])
        protein_id_list.append(i.qualifiers['protein_id'])
        translation_list.append(i.qualifiers['translation'])
        if 'product' in i.qualifiers.keys():
            product_list.append(i.qualifiers['product'])

if 'product' in i.qualifiers.keys():
   for i in zip(location_list, codon_start_list, product_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, product, protein_id, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{i[4][0]}", "{locus_accession}")'
        print(sql_feat_cds) 
        Cursor.execute(sql_feat_cds)
        DataBase.commit()
        
else:
    for i in zip(location_list, codon_start_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, protein_id, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{locus_accession}")'
        print(sql_feat_cds)
        Cursor.execute(sql_feat_cds)
        DataBase.commit()


INSERT INTO feat_cds (location, codon_start, protein_id, translation, locus_accession) VALUES ("[87:1176](+)", "1", "AAB59484.1", "MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPEYWDRNTQICKTNTQTDRESLRNLRGYYNQSEAGSHTWQTMYGCDVGPDGRLLRGHNQFAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQLRTYLEGTCVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAASSDSAQGSDVSLTA", "L42024.1")


### Tabela feat_gene ###

In [45]:
for i in features:
    if i.type == 'gene':
        
        gene_location_list = list()
        gene_name_list = list()

        gene_location_list.append(i.location)
        
        if 'locus_tag' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['locus_tag'])
        elif 'gene' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['gene'])

        if 'locus_tag' or 'gene' in i.qualifiers.keys():
            for i in zip(gene_location_list, gene_name_list):
                sql_feat_gene = f'INSERT INTO feat_gene (location, name, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()
                
        else:
            for i in gene_location_list:
                sql_feat_gene = f'INSERT INTO feat_gene (location, locus_accession) VALUES ("{i}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()

INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[0:1357](+)", "HLA-B39", "L42024.1")
